# Beer Reviews: Analysis and Recommendation

###  Recommend beers based on the reviews

#### Approach 1 - General Recommendation: Ranking of overall reviews after cleaning data

#### Approach 2 - Customized Recommendation: Content-based recommendation system

#### Data
1.5M beer reviews from beer advocate. (https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz)


In [2]:
%matplotlib inline

### Import packages Numpy, Scikit Learn and Pandas

In [3]:
import numpy as np
import sklearn
import pandas as pd

Read in the "csv" file with reviews into a pandas "data frame". Pandas has this nice reader that can read a bunch of file formats and store the data in a "data frame"

In [4]:
beerData = pd.read_csv('/Users/phani/Downloads/beer_reviews/beer_reviews.csv', delimiter=",", encoding='utf-8')
# download the revievs csv form the link above. Alternatively, use 'urlopen' and rebuild this command 
#to read the csv file after unzipping the response 

## Recommend beers based on the data - General recommendation

A simple ordering of data based on review_overall should give us a list with beers and their corresponding ranking in the list. 

#### Data Cleaning 
Number of reviews are not same for all beers. So, we will calculate the sample mean from the number of reviews we gathered for a beer to assign one overall_review for each beer. However, some beers have only one review where as others have more than one reviews. Hence, we need to clean up the data to include only those beers where we can calculate the mean within a certain margin of error. 

#### Choosing Sample Means: 
The statistics way to chose the threshold number of reviews (min number of samples) is to compute the minimum number of required reviews for a beer to predict the mean with 95% confidence interval. 

We use this formula: ($\frac{\sigma^2 * Z^2}{m^2}$), where $\sigma$ is the standard deviation of the sample, Z-score for a confidence interval of 95% is 1.96 and m is the allowed margin of error. 

In [5]:
# define a new dataframe with four attributes
samplesDF = beerData[["beer_beerid","beer_name","review_overall", "review_profilename"]]

# drop duplicate reviews for the same beer
samplesDF = samplesDF.drop_duplicates(["beer_beerid","review_profilename"])

# set indices for determining levels
samplesDF = samplesDF.set_index(["beer_beerid","beer_name"])

# Calculate nSamples, sampleMeans, sampleStdDev
nSamples = samplesDF.groupby(level=0).count().to_dict()
sampleMeans = samplesDF.groupby(level=0).mean().to_dict()
sampleStdDev = samplesDF.groupby(level=0).std()

# Define Margin of Error and Z-score for 95% confidence interval
mError = 0.1
zScore = 1.96

#### 1. filter out sampleMeans with less number of reviews than minimum required to achieve 95% confidence interval 
#### 2. sort sampleMeans and rank beer_ids from the sorted sampleMeans
#### 3. reject samples with std dev = 0.0

In [6]:
sampleMeansTemp = {}
for key in nSamples.keys(): 
    if key == "review_overall": # we are only interested in overall_review
        for beerID in nSamples[key].keys(): # get the values - beer_beerid and overall review
            if sampleStdDev[key][beerID] > 0:
                nSamplesRequired = (sampleStdDev[key][beerID] * zScore/mError)**2
            if nSamples[key][beerID] > nSamplesRequired:
                sampleMeansTemp[beerID] =  sampleMeans[key][beerID]

# redefine sampleMeans by sorted overall_reviews 
sampleMeans = sorted(sampleMeansTemp.items(), key=lambda x: x[1] , reverse=True)

1. Filter out the beerIDs that are not included in sampleMeans list
2. make a new dataframe 

Note: appending rows to make a new data frame takes a lot of time. 
3. So take the original data frame and drop the rows by comparing beerIDs

In [7]:
reviewBeerIDs = [beerKey[0] for beerKey in sampleMeans]
# drop the duplicate beerIDs 
newBeerDF = beerData.drop_duplicates(["beer_beerid"])
beerIDsAll = newBeerDF.beer_beerid.tolist()

# list the iDs that we need to discard
discardBeerIDs = [beerID for beerID in beerIDsAll if beerID not in reviewBeerIDs]
newBeerDF = newBeerDF.set_index(["beer_beerid"])
newBeerDF = newBeerDF.drop(discardBeerIDs)

#drop other labels and leave only few for visualization
newBeerDF = newBeerDF.drop(['brewery_id','review_time', 'review_overall','review_aroma','review_taste',
                    'review_palate','review_profilename','beer_abv','review_appearance'], axis=1)



In [8]:
# Create a column review_overall with values from sampleMeans
review_overall = []
for beerIndex in newBeerDF.index.tolist():
    for keyIndex in range(len(sampleMeans)):
        if sampleMeans[keyIndex][0] == beerIndex:
            review_overall.append(sampleMeans[keyIndex][1])

# add the column review_overall values from sampleMeans list
newBeerDF['review_overall'] = review_overall

#### Print the top ten beers in the list - General Recommendations

In [9]:
# sort the dataframe by overall reviews and print the top ten beers in the list
newBeerDF = newBeerDF.sort_values(by='review_overall', ascending=False)
newBeerDF.head(10)

,brewery_name,beer_style,beer_name,review_overall
beer_beerid,,,,
63649,Peg's Cantina & Brewpub / Cycle Brewing,American Double / Imperial Stout,Rare D.O.S.,4.848485
44910,De Struise Brouwers,Lambic - Unblended,Dirty Horse,4.820513
8626,Southampton Publick House,Berliner Weissbier,Southampton Berliner Weisse,4.768293
68548,Brouwerij Drie Fonteinen,Gueuze,Armand'4 Oude Geuze Lente (Spring),4.730769
70356,Brouwerij Drie Fonteinen,Gueuze,Armand'4 Oude Geuze Zomer (Summer),4.644444
56082,Kern River Brewing Company,American Double / Imperial IPA,Citra DIPA,4.628049
36316,Brasserie Cantillon,Lambic - Fruit,Cantillon Blåbær Lambik,4.625806
41928,Russian River Brewing Company,American Wild Ale,Deviation - Bottleworks 9th Anniversary,4.620536
16814,The Alchemist,American Double / Imperial IPA,Heady Topper,4.618510


## Content-based recommendation system - Customized Recommendations

In a content based recommendation system, we will compute preferences for each user (reviewer == user) and the  predict the rating for each beer (that the particular user has not rated before). Based on the predictions, we will recommend the beers that the user hasn't tried before and he may like based on his proferences.

#### Data Matrix: 

Rows are the beerIDs and the columns correspond to user ratings. Each cell in the matrix correspond to a beer and the overall rating for that beer by the user in that column. 

Since all users have not rated all beers, many of the cells are zero. 

#### Feature Matrix:
Feature matrix rows are beerIDs and columns are features. It contains four features - appearance,aroma, palate, taste. The data in the columns corresponding to a particular beerID is the samplemeans of each feature from all the ratings given for that beerID. We assume that the mean rating is proportional to the attributes of a given beer. (We will not use ABV% here as one of the feature. It has many undefined values that could shrink our feature set.)

#### Data Cleaning:
1. We collect user info based on review_profilename. Hence, some data points with missing "review_profilename" are dropped from the data matrix. 
2. We will use the reduced dataset from the above approach that satisfy minimum number of samples required to predict population means with 95% confidence level given a sample stdDev and margin of error. 
3. We will use sampleMeans for each attribute: aroma, palette, appearance and taste
4. Overall_reviews are the individual reviews given by a user

#### Build the Feature Matrix

In [10]:
# Feature Matrix
featureDF = beerData[["beer_beerid", "review_profilename",'review_appearance','review_aroma', 
                      'review_palate','review_taste','review_overall']]
featureDF = featureDF.drop_duplicates(["beer_beerid","review_profilename"])
featureDF = featureDF.set_index("beer_beerid")

# discard the beers that didn't meet our screening criterion of 95% confidence level
featureDF = featureDF.drop(discardBeerIDs)
featureDF = featureDF.reset_index()


# Make lists that match data matrix indices
beerIDList = sorted(featureDF.beer_beerid.unique())
profileList = featureDF.review_profilename.unique()

# Reindex the dataframe for extracting features.
featureDF = featureDF.set_index(["beer_beerid","review_profilename"])

#print(len(beerIDList),len(profileList))



In [11]:
# features sampleMeans
featuresDict = featureDF.groupby(level=0).mean().to_dict()
appearanceSampleMeans = featuresDict['review_appearance']
aromaSampleMeans = featuresDict['review_aroma']
palateSampleMeans = featuresDict['review_palate']
tasteSampleMeans = featuresDict['review_taste']

# Construct a numpy matrix with features Sample Means
featureMatrix = np.zeros(len(beerIDList*5)).reshape(len(beerIDList),5)
featuresMeansDicts = [appearanceSampleMeans,aromaSampleMeans,
                      palateSampleMeans,tasteSampleMeans]

# Populate the first element of the feature matrix with beerID
for beerIndex in range(len(beerIDList)):
    featureMatrix[beerIndex][0] = beerIDList[beerIndex]
    
featureIndex = 1 # feature index in feature Matrix
for featureDict in featuresMeansDicts:
    for beerIndex in range(len(beerIDList)):
        for key in featureDict.keys():
            if key == beerIDList[beerIndex]:
                featureMatrix[beerIndex][featureIndex] = featureDict[key]
    featureIndex += 1

# Add bias column in the featureMatrix
featureMatrix = np.insert(featureMatrix,1,1, axis=1)

#### Define Data Matrix and compute userPreferences

Constructing a data matrix with all users and all beers takes a lot of time and memory
We will take one user (or a block of users) at a time and evaluate the paramater matrix that correspond to the user preferences. 



In [12]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import timeit

#Create a dictionaryobject that holds users (profileNames) and 
# corresponding preference matrices
userPreferences = {}

# we will use this sampleMeans for mean normalization in Data Matrix
#sampleMeans = featuresDict['review_overall'] 


# For the total number of users in the list ~30000, it takes lot of time (> 5 hrs).
# time consuming constructs here:
# lookup in data frame is approximately 350us
# regression is between 400-700us
# generating X and y numpy arrays around 100us

# Generating user preferences for first 100 users
for profile in profileList[:10]:
    dataMatrix = np.zeros(len(beerIDList*2)).reshape(len(beerIDList),2)
    for beerIndex in range(len(beerIDList)):
        dataMatrix[beerIndex][0] = beerIDList[beerIndex]
        try:
            dataMatrix[beerIndex][1] = featureDF.loc[beerIDList[beerIndex],profile].tolist()[0]
        except KeyError:
            dataMatrix[beerIndex][1] = 0.0
            
            
    # subtract sample Means to do mean normalization
    # if dataMatrix[beerIndex][1] > 0:
    # for key in sampleMeans.keys():
    # if key == beerIDList[beerIndex]:
    # dataMatrix[beerIndex][1] -= sampleMeans[key]
                
    # X and y matrices for linear regression
    # Including all the rows resultd in poor R^2 values
    # Hence, only the rows with reviews are included in the fit
    # Bias term not included
    y = np.array([dataMatrix[i][1] for i in range(dataMatrix.shape[0]) if dataMatrix[i][1] > 0])
    X = np.array([featureMatrix[i][2:] for i in range(featureMatrix.shape[0]) if dataMatrix[i][1] > 0])

    
    # linear regression to compute parameter matrix
    regressor = LinearRegression()
    regressor.fit(X,y)
    userPreferences[profile] = [regressor.coef_,regressor.score(X,y)]


    # print('Weight coefficients: ', regressor.coef_)
    # print('y-axis intercept: ', regressor.intercept_)
    # print(regressor.score(X,y))
    
    
# Other possible regression options
    # KNeighbors Regressor
    # for i in range(1,10):
        # kneighbor_regression = KNeighborsRegressor(n_neighbors=i)
        # kneighbor_regression.fit(X, y)
        # print ("No of Neighbors: ", i)
        # print(kneighbor_regression.score(X, y))

    # Ridge Regreession. Includes Regularization (L2 penalty)
    # print ("Ridge Regression (L2 penalty)")
    # ridge_models = {} 
    # for alpha in [100, 10, 1, .01]:
        # ridge = Ridge(alpha=alpha).fit(X, y)
        # print("alpha = :", alpha)
        # print(ridge.score(X, y))
        # ridge_models[alpha] = ridge
        
    # print ("Lasso Regression (L1 penalty)")
    # lasso_models = {}
    #for alpha in [.01,0.001]:
        # lasso = Lasso(alpha=alpha).fit(X, y)
        # print("alpha = :", alpha)
        # print(lasso.score(X, y))
        # lasso_models[alpha] = lasso
        

#### Predicting the review based on featureMatrix and userPreferences

After populating the userPreferences dictionary with user preferences, 
for a given user, we can predict what his/her rating could be based on the feature matrix and his preferences

For Example, the 10th user in profileList has rated about 40 beers out of 2529 in the list. Based on our prediction, we will recommend other beers that he may like.

In [50]:
# user in the list of profiles
profileName  = profile

# List of beers that weren't rated by the user
beersNotRated = np.array([featureMatrix[i][0] for i in range(featureMatrix.shape[0])if dataMatrix[i][1] == 0])

# Feature list of beers
X_notRated = np.array([featureMatrix[i][2:] for i in range(featureMatrix.shape[0]) if dataMatrix[i][1] == 0])

# computed userPreferences from our regression analysis
userPref = userPreferences[profileName][0]
userPref = userPref[:,np.newaxis]

# Compute predicted ratings
y_predRatings = np.dot(X_notRated, userPref)
# We will cap the rating at 5.0 Our regression analysis has predicted values over 5
for i in range(y_predRatings.shape[0]):
    if y_predRatings[i] > 5.0:
        y_predRatings[i] = 5.0

# prepare a new dataFrame to display the top recommendations
predDF = pd.DataFrame()
# Add two columns
predDF['beerID'] = [int(beersNotRated[i]) for i in range(beersNotRated.shape[0])]
predDF['predRating'] = [y_predRatings[i][0] for i in range(beersNotRated.shape[0])]
# Sort in descending order. Main column is the predicted overall rating
predDF.sort_values(by='predRating', ascending=False, inplace=True)

#Extract beerIDs to collect beerName and beerStyle information from original dataFrame 
predBeerIDs = predDF.beerID.tolist() 

# Add other columns to the recommendation dataframe
avgAppearance = [appearanceSampleMeans[i] for i in predBeerIDs]
avgAroma = [aromaSampleMeans[i] for i in predBeerIDs]
avgPalate = [palateSampleMeans[i] for i in predBeerIDs]
avgTaste = [tasteSampleMeans[i] for i in predBeerIDs]
beerNames = [beerData[beerData.beer_beerid == i].beer_name.tolist()[0] for i in predBeerIDs]
breweryNames = [beerData[beerData.beer_beerid == i].brewery_name.tolist()[0] for i in predBeerIDs]
beerStyles = [beerData[beerData.beer_beerid == i].beer_style.tolist()[0] for i in predBeerIDs]
predDF['Aroma'] = avgAroma
predDF['Appearance'] = avgAppearance
predDF['Palate'] = avgPalate
predDF['Taste'] = avgTaste
predDF['BeerName'] = beerNames
predDF['BreweryName'] = breweryNames
predDF['BeerStyle'] = beerStyles

### Our top recommendations for the selected user

In [51]:
print ("ProfileName: ", profileName)
topRecommendations = predDF.head(10).set_index("BeerName")
topRecommendations.drop("beerID", axis=1, inplace=True)
topRecommendations

ProfileName:  mikesgroove


,predRating,Aroma,Appearance,Palate,Taste,BreweryName,BeerStyle
BeerName,,,,,,,
Dirty Horse,5.000000,4.615385,4.423077,4.576923,4.743590,De Struise Brouwers,Lambic - Unblended
Cable Car Kriek,5.000000,4.600000,4.500000,4.450000,4.750000,The Lost Abbey,American Wild Ale
Bitter Monk,4.875383,4.292035,4.309735,4.159292,4.323009,Anchorage Brewing Company,Belgian IPA
Saison Deluxe,4.808750,4.128505,4.345794,4.137850,4.231308,Southampton Publick House,Saison / Farmhouse Ale
Hill Farmstead Double Citra IPA,4.801457,4.349057,4.207547,4.264151,4.386792,Hill Farmstead Brewery,American Double / Imperial IPA
Unibroue 17,4.794529,4.175926,4.277778,4.092593,4.205556,Unibroue,Belgian Strong Dark Ale
Anna,4.786022,4.115385,4.317308,4.076923,4.105769,Hill Farmstead Brewery,Saison / Farmhouse Ale
Bell's Batch 6000,4.782734,4.387879,4.193939,4.372727,4.430303,"Bell's Brewery, Inc.",American Barleywine
Baudelaire IO,4.772817,4.108527,4.267442,3.953488,4.007752,Jolly Pumpkin Artisan Ales,Saison / Farmhouse Ale


In [ ]:
print ("Number of Samples: " , featureMatrix.shape[0])
print ("Number of Features: ", featureMatrix.shape[1])
#print (dataMatrix.shape[0])

X = np.array([featureMatrix[i][1:] for i in range(featureMatrix.shape[0])if dataMatrix[i][1] > 0])
y = np.array([dataMatrix[i][1] for i in range(dataMatrix.shape[0])if dataMatrix[i][1] > 0])

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
xT = X.T

x1 = xT[0]
x2 = xT[1]
x3 = xT[2]
x4 = xT[3]

#plt.plot(xT[3],y, 'o');
plt.subplot(2, 2, 1)
plt.plot(x1, y, 'o')
plt.ylabel('Appearance')

plt.subplot(2, 2, 2)
plt.plot(x2, y, 'o')
plt.ylabel('Aroma')

plt.subplot(2, 2, 3)
plt.plot(x3, y, 'o')
plt.ylabel('Palate')

plt.subplot(2, 2, 4)
plt.plot(x4, y, 'o')
plt.ylabel('Taste')

In [ ]:
import sklearn 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, random_state=1236)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(train_X, train_y)
print('Weight coefficients: ', regressor.coef_)
print('y-axis intercept: ', regressor.intercept_)
print(regressor.score(train_X,train_y))
print(regressor.score(test_X,test_y))

y_pred_train = regressor.predict(train_X)
y_pred_test = regressor.predict(test_X)

plt.plot([y.min(), y.max()], [y.min(), y.max()])
plt.plot(test_y, y_pred_test, '^')
plt.xlabel('Actual Review')
plt.ylabel('Predicted Review')

print ("Lasso Regression (L1 penalty)")
from sklearn.linear_model import Lasso
lasso_models = {}
for alpha in [100, 10, 1, .01]:
    lasso = Lasso(alpha=alpha).fit(train_X, train_y)
    print("alpha = :", alpha)
    print(lasso.score(train_X, train_y))
    print(lasso.score(test_X, test_y))
    lasso_models[alpha] = lasso

print ("Ridge Regression (L2 penalty)")
from sklearn.linear_model import Ridge
ridge_models = {}
for alpha in [100, 10, 1, .01]:
    ridge = Ridge(alpha=alpha).fit(train_X, train_y)
    print("alpha = :", alpha)
    print(ridge.score(train_X, train_y))
    print(ridge.score(test_X, test_y))
    ridge_models[alpha] = ridge

from sklearn.neighbors import KNeighborsRegressor
for i in range(1,5):
    kneighbor_regression = KNeighborsRegressor(n_neighbors=i)
    kneighbor_regression.fit(train_X, train_y)
    print ("No of Neighbors: ", i)
    print(kneighbor_regression.score(train_X, train_y))
    print(kneighbor_regression.score(test_X, test_y))




#### Build a content-based recommendation system for each user

For a reviewer, we will recommend beers that he might like based on the data from his own ratings of other beers and sampleMeans of features for all beers.

Data Matrix: DataMatrix contains the overall_review for all beers separated by each reviewer along columns. (Difference between this and above is that in the above approach, sampleMeans of overall reviews are listed in dataMatrix) 

Features Matrix: Features matrix is same as above

Finally, we use linear regression to calculate the parameters for each user that correlate features with overall rating. 

In [ ]:
# Data Matrix
dataDF = beerData[["beer_beerid","review_overall", "review_profilename"]]

# drop duplicate reviews for the same beer
dataDF = dataDF.drop_duplicates(["beer_beerid","review_profilename"])

# set indices for determining levels
dataDF = dataDF.set_index("beer_beerid")

# drop the rows with less number of reviews than required samples
# Refer to diff variable from above
dataDF = dataDF.drop(discardBeerIDs)

# Make a numpy 2D array with beerIDs on one axis 
# and profilenames on the other axis
dataDF = dataDF.reset_index()
beerIDList = sorted(dataDF.beer_beerid.unique())
profileList = dataDF.review_profilename.unique()

#************************************
# Building an actual data matrix with all beers and review_profiles is time consuming
# The code below shows the implementation and test for 2 beers and validate the approach.

# Define Data Matrix
dataMatrix = np.zeros(len(beerIDList) * len(profileList)).reshape(len(beerIDList),len(profileList))

# Reindex to use access dataframe using two indices of numpy array
dataDF = dataDF.set_index(["beer_beerid","review_profilename"])

for beerID in range(2):
    for profile in range(len(profileList)):
        try:
            rating = dataDF.loc[(beerIDList[beerID],profileList[profile]),"review_overall"]
            dataMatrix[beerID][profile] = rating
            #print(profile, beerID, rating)
        except KeyError:
            pass
    
            
#print (len([dataMatrix[0][i] for i in range(len(dataMatrix[0])) if dataMatrix[0][i] > 0]))

#### Calculate the parameterMatrix based on featureMatrix and subsets of dataMatrix (Incomplete!)

In [ ]:
# Take first ten profile names and build a dataMatrix for 10 profiles
profiles = profileList[:10]

# Define Data Matrix
subDataMatrix = np.zeros(len(beerIDList) * (len(profiles) + 1)).reshape(len(beerIDList),len(profiles)+1)

# Reindex to use access dataframe using two indices of numpy array
#dataDF = dataDF.set_index(["beer_beerid","review_profilename"])

for beerIndex in range(len(beerIDList)):
    subDataMatrix[beerIndex][0] = beerIDList[beerIndex]
    
for beerID in range(len(beerIDList)):
    for profile in range(len(profiles)):
        try:
            rating = dataDF.loc[(beerIDList[beerID],profiles[profile]),"review_overall"]
            subDataMatrix[beerID][profile+1] = rating
        except KeyError:
            pass

print (np.shape(subDataMatrix))
print (np.shape(featureMatrix))
